In [6]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# 경로 설정
keypoints_dir = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final\keypoints3d"
save_path = r"C:\Users\SongYoengEun\Desktop\cap\Motion_Metadata_with_Emotions.csv"

# 관절 인덱스
left_arm = [5, 7, 9]
right_arm = [6, 8, 10]
pelvis_indices = [11, 12]

results = []

def extract_features(pose3d):
    arm_y = pose3d[:, left_arm + right_arm, 1].mean(axis=1)
    if len(arm_y) > 30:
        arm_delta = arm_y[30:] - arm_y[:-30]
        avg_arm_delta = arm_delta.mean()
    else:
        avg_arm_delta = 0

    speed = np.linalg.norm(np.diff(pose3d, axis=0), axis=2).mean()
    pelvis_center = pose3d[:, pelvis_indices, :].mean(axis=1)
    pelvis_shift = np.linalg.norm(np.diff(pelvis_center, axis=0), axis=1).mean()

    spread = []
    for frame in pose3d:
        dists = [np.linalg.norm(frame[i] - frame[j]) for i in range(17) for j in range(i+1, 17)]
        spread.append(np.std(dists))
    avg_spread = np.mean(spread)

    return avg_arm_delta, speed, pelvis_shift, avg_spread

def predict_emotions(avg_arm_delta, speed, pelvis_shift, avg_spread):
    emotions = []

    if avg_arm_delta > 15:
        emotions += ["자신감", "행복"]
    elif avg_arm_delta < -15:
        emotions += ["슬픔"]

    if speed > 20:
        emotions += ["열정"]
    elif speed > 12:
        emotions += ["희망"]
    elif speed < 8:
        emotions += ["차분함"]

    if pelvis_shift > 10:
        emotions += ["도전"]
    elif pelvis_shift < 5:
        emotions += ["슬픔", "차분함"]

    if avg_spread > 140:
        emotions += ["매혹"]
    elif avg_spread < 90:
        emotions += ["공포"]

    emotion_counter = Counter(emotions)
    top_emotions = [e for e, _ in emotion_counter.most_common(2)]
    return top_emotions

for filename in os.listdir(keypoints_dir):
    if not filename.endswith(".pkl"):
        continue
    motion_id = filename.replace(".pkl", "")
    with open(os.path.join(keypoints_dir, filename), "rb") as f:
        data = pickle.load(f)
        if "keypoints3d_optim" not in data:
            continue
        pose3d = data["keypoints3d_optim"]

    avg_arm_delta, speed, pelvis_shift, avg_spread = extract_features(pose3d)
    emotions = predict_emotions(avg_arm_delta, speed, pelvis_shift, avg_spread)

    results.append({
        "MOTION_ID": motion_id,
        "EMOTION_1": emotions[0] if len(emotions) > 0 else None,
        "EMOTION_2": emotions[1] if len(emotions) > 1 else None,
        "avg_arm_delta": avg_arm_delta,
        "speed": speed,
        "pelvis_shift": pelvis_shift,
        "avg_spread": avg_spread
    })

# 저장
if results:
    df = pd.DataFrame(results)
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print("감정 예측 결과 저장 완료!")
    print(df.head())
else:
    print("분석 결과 없음!")


✅ 감정 예측 결과 저장 완료!
                    MOTION_ID EMOTION_1 EMOTION_2  avg_arm_delta     speed  \
0  gBR_sBM_cAll_d04_mBR0_ch01       차분함        슬픔       0.442163  1.190426   
1  gBR_sBM_cAll_d04_mBR0_ch02       차분함        슬픔       0.060897  1.730326   
2  gBR_sBM_cAll_d04_mBR0_ch03       차분함        슬픔      -0.184639  1.417298   
3  gBR_sBM_cAll_d04_mBR0_ch04       차분함        슬픔       0.050326  1.656661   
4  gBR_sBM_cAll_d04_mBR0_ch05       차분함        슬픔      -0.938572  1.717238   

   pelvis_shift  avg_spread  
0      0.778513   36.385095  
1      1.017101   34.046623  
2      1.030659   35.658800  
3      0.842656   33.999885  
4      0.981873   31.838545  
